<a href="https://colab.research.google.com/github/swopnimghimire-123123/Machine-Learning-Journey/blob/main/28_Column_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
df = pd.read_csv("/content/covid_toy.csv")

In [ ]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
18,64,Female,98.0,Mild,Bangalore,Yes
34,74,Male,102.0,Mild,Mumbai,Yes
88,5,Female,100.0,Mild,Kolkata,No
36,38,Female,101.0,Mild,Bangalore,No
25,23,Male,NaN,Mild,Mumbai,No


In [ ]:
df.isnull().sum()
# It's to be noted that there are 10 null values in fever

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),
                                                 df['has_covid'],
                                                 test_size=0.2)

In [ ]:
X_train.sample(3)

,age,gender,fever,cough,city
34,74,Male,102.0,Mild,Mumbai
78,11,Male,100.0,Mild,Bangalore
2,42,Male,101.0,Mild,Delhi


###**Normal Approach which we have studied till now**

In [ ]:
# Adding SimpleImputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
X_train_fever.shape


(80, 1)

In [ ]:
# OrdinalEncoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])
X_train.shape

(80, 5)

In [ ]:
# OneHotEncoding -> gender,city
OHE = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = OHE.fit_transform(X_train[['gender','city']])

# also the test dat
X_test_gender_city = OHE.fit_transform(X_test[['gender','city']])
X_train_gender_city.shape

(80, 4)

In [ ]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(80, 1)

In [ ]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)
X_train_transformed.shape

(80, 7)

###**COLUMN TRANSFORMER**

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder="passthrough")

In [ ]:
transformer.fit_transform(X_train).shape

(80, 7)

In [ ]:
transformer.fit_transform(X_test).shape

(20, 7)